# **Section 1: Environment Setup & Dependencies**
In this section, we configure the runtime environment, mount persistent storage (Google Drive), and authenticate with Hugging Face to access the required datasets.

### **1. Project Setup & Directory Initialization**
We initialize the project structure by creating the necessary subdirectories for raw data, processed features, metadata, and model checkpoints.

In [ ]:
!mkdir -p /content/drive/MyDrive/hindi_dfake/{raw,processed,metadata,scripts,checkpoints}

### **2. Environment Configuration**
We mount Google Drive to ensure persistent storage and dynamically define the root paths (`ROOT_DIR`, `OUT_DIR`, etc.) to avoid hardcoding errors across different sessions.

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

import os, glob
MOUNT_PATH = "/content/drive" if os.path.isdir("/content/drive/MyDrive") else "/content/gdrive"
print("Drive is already mounted here →", MOUNT_PATH)
ROOT_DIR = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR  = f"{ROOT_DIR}/raw/real_clean/ivr"
CSV_PATH = f"{ROOT_DIR}/metadata/master_real.csv"

Mounted at /content/drive
Drive is already mounted here → /content/drive


### **3. Hugging Face Authentication**
Authentication is required to download the **IndicVoices-R** and **Common Voice** datasets directly from the Hugging Face Hub.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# **Section 2: Simulated Attacks**
We apply codec compression (Opus 16k) and channel effects (Low-pass 8kHz) to a 40% subset of the data. This ensures the detector works on compressed, real-world audio.

### **1. Attack Dataset Selection**
We select the files to be processed. You can adjust `REAL_FRACTION` and `FAKE_FRACTION` if you want to generate a smaller debug set first, or set them to `1.0` to process the entire dataset.

In [ ]:
# ===== Codec/Channel Attacks — DEBUG Selection (fast + verbose) =====
# Real: raw/real_clean/{commonvoice, indictts, ivr}
# Fake: raw/fake_tts

# Paths
ROOT = "/content/drive/MyDrive/hindi_dfake"
META = f"{ROOT}/metadata"
REAL_CSV   = f"{META}/master_real.csv"
FAKE_CSV   = f"{META}/master_fake.csv"
OUT_DIR    = f"{ROOT}/processed/attacks"   # next cells will write here

# Include both classes
INCLUDE_REAL = True
INCLUDE_FAKE = True

# Optional sampling to control runtime/storage
REAL_FRACTION = 1.0   # try 0.10 first if you want to smoke-test
FAKE_FRACTION = 1.0

# Attack list is defined in the next cells; we just select rows here.

# --- DEBUG switches ---
PROBE_AUDIO_HEADERS = False   # set True to verify a small sample with sf.info
PROBE_LIMIT_PER_SRC = 100     # header-probe at most this many files per source

# --- deps ---
!pip -q install pandas soundfile numpy > /dev/null

import os, time, pandas as pd, numpy as np, soundfile as sf
from pathlib import Path
from collections import Counter

start = time.time()
def log(msg):
    print(f"[+{time.time()-start:6.1f}s] {msg}", flush=True)

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# 0) File presence
for p in [REAL_CSV, FAKE_CSV]:
    if os.path.exists(p):
        log(f"Found {p} ({os.path.getsize(p)/1e6:.2f} MB)")
    else:
        log(f"Missing: {p}")

# 1) Load minimal columns quickly
USECOLS = ["utt_id","path","duration","speaker_id","gender","text","label"]
log("Reading CSVs...")
dfr = pd.read_csv(REAL_CSV, usecols=[c for c in USECOLS if c != "label"], low_memory=False) if INCLUDE_REAL and os.path.exists(REAL_CSV) else pd.DataFrame(columns=USECOLS)
dff = pd.read_csv(FAKE_CSV, usecols=USECOLS, low_memory=False) if INCLUDE_FAKE and os.path.exists(FAKE_CSV) else pd.DataFrame(columns=USECOLS)
if "label" not in dfr.columns and len(dfr):
    dfr["label"] = "real"
log(f"Loaded rows — real: {len(dfr):,} | fake: {len(dff):,}")

# 2) Filter by folder (no regex groups; no warnings)
def keep_real_folders(df):
    p = df["path"].astype(str)
    mask = (
        p.str.contains("/raw/real_clean/commonvoice", regex=False, na=False) |
        p.str.contains("/raw/real_clean/indictts",   regex=False, na=False) |
        p.str.contains("/raw/real_clean/ivr",        regex=False, na=False)
    )
    return df[mask].copy()

def keep_fake_folder(df):
    p = df["path"].astype(str)
    mask = p.str.contains("/raw/fake_tts", regex=False, na=False)
    return df[mask].copy()

log("Filtering rows to target folders...")
dfr = keep_real_folders(dfr) if len(dfr) else dfr
dff = keep_fake_folder(dff)  if len(dff) else dff
log(f"After folder filter — real: {len(dfr):,} | fake: {len(dff):,}")

# 3) Fast existence check (no audio decode)
def fast_exists(paths):
    out = np.zeros(len(paths), dtype=bool)
    for i, p in enumerate(paths):
        out[i] = Path(p).exists()
        if (i+1) % 10000 == 0:
            log(f"exists() checked: {i+1}/{len(paths)}")
    return out

if len(dfr):
    log("exists() on real...")
    m = fast_exists(dfr["path"].astype(str).tolist())
    dfr = dfr[m]
    log(f"Real kept after exists(): {len(dfr):,}")
if len(dff):
    log("exists() on fake...")
    m = fast_exists(dff["path"].astype(str).tolist())
    dff = dff[m]
    log(f"Fake kept after exists(): {len(dff):,}")

# 4) Optional: header-only probe (sampled) — NOT full decode
if PROBE_AUDIO_HEADERS:
    log("Header-probing a small sample with soundfile.info()...")
    def probe_sample(df, name):
        samp = df.sample(min(len(df), PROBE_LIMIT_PER_SRC), random_state=2025) if len(df) else df
        sr_ct = Counter(); ch_ct = Counter()
        bad = 0
        for p in samp["path"].astype(str):
            try:
                info = sf.info(p)
                sr_ct[info.samplerate] += 1
                ch_ct[info.channels]   += 1
            except Exception:
                bad += 1
        log(f"{name}: sample={len(samp)} | sr={dict(sr_ct)} | ch={dict(ch_ct)} | bad={bad}")
    probe_sample(dfr, "real")
    probe_sample(dff, "fake")

# 5) Optional sampling to control runtime/storage
if 0 < REAL_FRACTION < 1.0 and len(dfr):
    before = len(dfr); dfr = dfr.sample(frac=REAL_FRACTION, random_state=2025); log(f"Sampled real: {before:,} -> {len(dfr):,}")
if 0 < FAKE_FRACTION < 1.0 and len(dff):
    before = len(dff); dff = dff.sample(frac=FAKE_FRACTION, random_state=2025); log(f"Sampled fake: {before:,} -> {len(dff):,}")

# 6) Tag sources for reporting
def src_from_path(p: str) -> str:
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    return "other"

if len(dfr): dfr["source"] = dfr["path"].astype(str).map(src_from_path)
if len(dff): dff["source"] = dff["path"].astype(str).map(src_from_path)

# 7) Combine and show a tiny preview
df_all = pd.concat([dfr, dff], ignore_index=True)
log(f"Selected inputs total: {len(df_all):,}")
if len(df_all):
    log("Source breakdown:\n" + df_all["source"].value_counts().to_string())
    log("Preview paths:\n" + "\n".join(df_all["path"].astype(str).head(5).tolist()))
else:
    log("No inputs selected — check your paths in master_* CSVs.")

[+   1.0s] Found /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv (10.29 MB)
[+   1.0s] Found /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv (9.65 MB)
[+   1.0s] Reading CSVs...
[+   2.5s] Loaded rows — real: 25,965 | fake: 23,731
[+   2.5s] Filtering rows to target folders...
[+   2.5s] After folder filter — real: 25,965 | fake: 23,731
[+   2.5s] exists() on real...
[+  40.9s] exists() checked: 10000/25965
[+  43.8s] exists() checked: 20000/25965
[+  45.7s] Real kept after exists(): 25,965
[+  45.7s] exists() on fake...
[+ 123.6s] exists() checked: 10000/23731
[+ 126.4s] exists() checked: 20000/23731
[+ 127.4s] Fake kept after exists(): 23,731
[+ 127.5s] Selected inputs total: 49,696
[+ 127.5s] Source breakdown:
source
fake_tts            23731
real_ivr            11174
real_commonvoice    10088
real_indictts        4703
[+ 127.5s] Preview paths:
/content/drive/MyDrive/hindi_dfake/raw/real_clean/ivr/1a0b61275bc58baf.wav
/content/drive/MyDrive/hindi_dfake/raw/re

### **2. Attack Generation**
We apply **Opus compression** (16k) and **Low-pass filtering** (8kHz) to the selected subset (40%) of real and fake files. This process is parallelized and resumable.

In [ ]:
# ===== Codec/Channel Attacks — subset + resumable + no-dup + progress =====
import os, time, csv, hashlib, subprocess, tempfile
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed

ROOT = "/content/drive/MyDrive/hindi_dfake"
META = Path(ROOT) / "metadata"
OUT_DIR = Path(ROOT) / "processed" / "attacks"
ATTACK_CSV = META / "attacks.csv"

# --- attacks (you can add more later) ---
ATTACKS = {
    "opus16": {"codec": "libopus",           "bitrate": "16k",  "container": "ogg"},
    "lp8k":   {"filter": "lowpass=f=8000"},
    # "opus24": {"codec": "libopus",           "bitrate": "24k",  "container": "ogg"},
    # "opus32": {"codec": "libopus",           "bitrate": "32k",  "container": "ogg"},
    # "amrnb":  {"codec": "libopencore_amrnb", "bitrate": "12.2k","ar": 8000, "container": "amr"},
}
ENABLED_ATTACKS = list(ATTACKS.keys())

# --- subset knobs (40% like you asked) ---
FRACTION_REAL = 0.40
FRACTION_FAKE = 0.40
HASH_SEED     = 2025    # stable selection across runs (no randomness drift)

MAX_WORKERS = 6
BATCH_WRITE = 1000      # write metadata every N successes
BACKFILL_METADATA = True  # if output file exists but not in CSV, add a row for it

assert 'df_all' in globals() and len(df_all) > 0, "Run the selection DEBUG cell first; it defines df_all."

# ---------- helpers ----------
def ffmpeg_supports(codec_name: str) -> bool:
    try:
        out = subprocess.run(["ffmpeg","-hide_banner","-codecs"],
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                             text=True, check=True).stdout
        return codec_name in out
    except Exception:
        return False

def run_ffmpeg(cmd_list):
    p = subprocess.run(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.returncode != 0:
        raise RuntimeError(p.stderr.decode("utf-8","ignore"))

def out_path_for(in_path: str, attack_key: str) -> Path:
    return OUT_DIR / attack_key / f"{Path(in_path).stem}.{attack_key}.wav"

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def wav_duration(p: Path) -> float:
    try:
        d, sr = sf.read(str(p), dtype="float32", always_2d=False)
        return len(d)/float(sr)
    except: return 0.0

def opus_roundtrip(in_wav: str, out_wav: str, bitrate: str):
    with tempfile.TemporaryDirectory() as td:
        ogg = str(Path(td)/"tmp.ogg")
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",in_wav,
                    "-ac","1","-ar","16000","-c:a","libopus","-b:a",bitrate, ogg])
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",ogg,
                    "-ac","1","-ar","16000","-sample_fmt","s16", out_wav])

def amrnb_roundtrip(in_wav: str, out_wav: str, bitrate: str, ar=8000):
    with tempfile.TemporaryDirectory() as td:
        amr = str(Path(td)/"tmp.amr")
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",in_wav,
                    "-ac","1","-ar",str(ar),"-c:a","libopencore_amrnb","-b:a",bitrate, amr])
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",amr,
                    "-ac","1","-ar","16000","-sample_fmt","s16", out_wav])

def filter_only(in_wav: str, out_wav: str, af: str):
    run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",in_wav,
                "-ac","1","-ar","16000","-af",af,"-sample_fmt","s16", out_wav])

def stable_frac_mask(series: pd.Series, frac: float, seed: int) -> pd.Series:
    """Deterministic subset by hashing path; same result every run."""
    if frac >= 1.0: return pd.Series([True]*len(series), index=series.index)
    # make 0..1 score from sha1(path + seed)
    s = series.astype(str).apply(lambda p: int(hashlib.sha1((p+str(seed)).encode()).hexdigest()[:8], 16) / 0xFFFFFFFF)
    return s < frac

# ---------- build subset (stratified by label & source, deterministic) ----------
def stratified_subset_hash(df):
    parts = []
    for (lab, src), g in df.groupby(["label","source"], dropna=False):
        frac = FRACTION_REAL if lab == "real" else FRACTION_FAKE
        if frac >= 1.0 or len(g) <= 1:
            parts.append(g)
        else:
            mask = stable_frac_mask(g["path"], frac, HASH_SEED)
            parts.append(g[mask])
    return pd.concat(parts, ignore_index=True) if parts else df

df_input = stratified_subset_hash(df_all.copy())
print("Subset sizes by label/source:")
print(df_input.groupby(["label","source"]).size().to_string())

# ---------- skip work we already did ----------
# paths already listed in attacks.csv
existing_paths_csv = set()
if ATTACK_CSV.exists():
    try:
        existing_paths_csv = set(pd.read_csv(ATTACK_CSV, usecols=["path"])["path"].astype(str))
    except Exception:
        existing_paths_csv = set()

# filter attacks by codec availability
enabled = []
for k in ENABLED_ATTACKS:
    spec = ATTACKS[k]
    if "codec" in spec and not ffmpeg_supports(spec["codec"]):
        print(f"[WARN] Disabling {k}: ffmpeg missing {spec['codec']}")
        continue
    enabled.append(k)
ENABLED_ATTACKS = enabled
print("Attacks enabled:", ENABLED_ATTACKS)

# ensure metadata exists with header for incremental appends
ATTACK_CSV.parent.mkdir(parents=True, exist_ok=True)
if not ATTACK_CSV.exists():
    with open(ATTACK_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["parent_utt","path","duration","speaker_id","gender","text","label",
                    "fake_type","attack","codec","bitrate","source"])

def append_rows(rows):
    if not rows: return
    with open(ATTACK_CSV, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["parent_utt","path","duration","speaker_id","gender","text","label",
                                          "fake_type","attack","codec","bitrate","source"])
        for r in rows:
            w.writerow(r)

# ---------- run per attack with explicit GOAL/DONE/REMAINING ----------
for atk in ENABLED_ATTACKS:
    spec = ATTACKS[atk]

    # planned outputs for our subset
    planned = df_input["path"].astype(str).apply(lambda p: str(out_path_for(p, atk)))
    planned_set = set(planned)

    # what's already on disk (for this attack)
    out_dir = OUT_DIR / atk
    out_dir.mkdir(parents=True, exist_ok=True)
    on_disk = set(str(p) for p in out_dir.glob("*.wav"))

    # already done = on disk (we treat as done even if CSV missed a row; we'll backfill)
    already_done = planned_set & on_disk
    # remaining = planned - on_disk
    remaining_set = planned_set - on_disk

    goal = len(planned_set)
    done = len(already_done)
    todo = len(remaining_set)
    print(f"\n[{atk}] GOAL={goal:,} | ALREADY_DONE(on-disk)={done:,} | TO_DO={todo:,}")

    # (optional) backfill CSV rows for outputs that exist but are missing from CSV
    if BACKFILL_METADATA:
        missing_meta = [p for p in already_done if p not in existing_paths_csv]
        if missing_meta:
            print(f"[{atk}] Backfilling metadata for {len(missing_meta):,} existing files...")
            rows = []
            # build a map from input path -> parent row for quick lookup
            in2row = {str(r["path"]): r for _, r in df_input.iterrows()}
            for out_p in missing_meta:
                # reconstruct parent input path from filename stem (before .atk.wav) by searching
                stem = Path(out_p).name.split(f".{atk}.wav")[0]
                # find any input whose stem matches (fast check)
                # this is approximate but works because we preserve input stem in output name
                cand = [r for r in in2row.values() if Path(str(r["path"])).stem == stem]
                base = cand[0] if cand else None
                dur = wav_duration(Path(out_p))
                rows.append({
                    "parent_utt": "" if base is None else base.get("utt_id",""),
                    "path": out_p,
                    "duration": round(dur,3),
                    "speaker_id": "" if base is None else base.get("speaker_id",""),
                    "gender": "" if base is None else base.get("gender",""),
                    "text": "" if base is None else base.get("text",""),
                    "label": "" if base is None else base.get("label",""),
                    "fake_type": "channel_attack",
                    "attack": atk,
                    "codec": spec.get("codec", spec.get("filter","")),
                    "bitrate": spec.get("bitrate",""),
                    "source": "" if base is None else base.get("source",""),
                })
            append_rows(rows)
            existing_paths_csv.update(missing_meta)
            print(f"[{atk}] Backfill appended.")

    if todo == 0:
        print(f"[{atk}] Nothing to do.")
        continue

    # build rows to process (Series aligned with df_input)
    to_do_mask = planned.isin(list(remaining_set))
    df_todo = df_input[to_do_mask].copy()
    n = len(df_todo)
    print(f"[{atk}] Will process {n:,} new files…")

    created = 0; errs = 0
    buffer_rows = []

    def apply_one(row):
        in_wav = str(row["path"])
        out_wav = str(out_path_for(in_wav, atk))
        outp = Path(out_wav)
        ensure_parent(outp)
        try:
            if spec.get("codec") == "libopus":
                opus_roundtrip(in_wav, out_wav, spec["bitrate"])
            elif spec.get("codec") == "libopencore_amrnb":
                amrnb_roundtrip(in_wav, out_wav, spec["bitrate"], spec.get("ar", 8000))
            elif "filter" in spec:
                filter_only(in_wav, out_wav, spec["filter"])
            else:
                raise ValueError(f"Unknown attack spec: {spec}")
        except Exception as e:
            try:
                if outp.exists(): outp.unlink()
            except: pass
            return {"error": str(e), "path": in_wav}

        dur = wav_duration(outp)
        if dur < 0.2:
            try: outp.unlink()
            except: pass
            return {"error": "too_short", "path": in_wav}

        return {
            "parent_utt": row.get("utt_id",""),
            "path": out_wav,
            "duration": round(dur, 3),
            "speaker_id": row.get("speaker_id",""),
            "gender": row.get("gender",""),
            "text": row.get("text",""),
            "label": row.get("label",""),
            "fake_type": "channel_attack",
            "attack": atk,
            "codec": spec.get("codec", spec.get("filter","")),
            "bitrate": spec.get("bitrate",""),
            "source": row.get("source",""),
        }

    t0 = time.time()
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs = [ex.submit(apply_one, r) for _, r in df_todo.iterrows()]
        done_ct = 0
        for f in as_completed(futs):
            done_ct += 1
            res = f.result()
            if isinstance(res, dict) and "error" in res:
                errs += 1
            else:
                buffer_rows.append(res)
                created += 1

            # periodic progress & metadata flush
            if done_ct % 1000 == 0 or (done_ct == n):
                print(f"[{atk}] {done_ct:,}/{n:,}  (+{created} ok, {errs} err)  | elapsed {time.time()-t0:.1f}s", flush=True)

            if len(buffer_rows) >= BATCH_WRITE or (done_ct == n and buffer_rows):
                append_rows(buffer_rows)
                existing_paths_csv.update([r["path"] for r in buffer_rows])
                buffer_rows.clear()

    print(f"[{atk}] finished: +{created} new, {errs} errors | total elapsed {time.time()-t0:.1f}s")

# quick per-attack summary (files/hours on disk)
def folder_hours(d: Path):
    s = 0.0
    for p in d.glob("*.wav"):
        try:
            x, sr = sf.read(str(p)); s += len(x)/float(sr)
        except: pass
    return s/3600

tot_files = 0; tot_hours = 0.0
for atk in ENABLED_ATTACKS:
    d = OUT_DIR / atk
    if not d.exists(): continue
    n = len(list(d.glob("*.wav")))
    h = folder_hours(d)
    tot_files += n; tot_hours += h
    print(f"[sum] {atk:6s} → files: {n:7d} | hours: {h:7.2f} h")
print(f"[sum] TOTAL  → files: {tot_files:7d} | hours: {tot_hours:7.2f} h")

Subset sizes by label/source:
label     source          
fake      fake_tts            8667
real      real_commonvoice    4022
          real_indictts       1833
          real_ivr            4481
tts_edge  fake_tts             900
Attacks enabled: ['opus16', 'lp8k']

[opus16] GOAL=19,903 | ALREADY_DONE(on-disk)=19,903 | TO_DO=0
[opus16] Nothing to do.

[lp8k] GOAL=19,903 | ALREADY_DONE(on-disk)=13,158 | TO_DO=6,745
[lp8k] Backfilling metadata for 158 existing files...
[lp8k] Backfill appended.
[lp8k] Will process 6,745 new files…
[lp8k] 1,000/6,745  (+1000 ok, 0 err)  | elapsed 146.9s
[lp8k] 2,000/6,745  (+2000 ok, 0 err)  | elapsed 293.6s
[lp8k] 3,000/6,745  (+3000 ok, 0 err)  | elapsed 440.5s
[lp8k] 4,000/6,745  (+4000 ok, 0 err)  | elapsed 587.1s
[lp8k] 5,000/6,745  (+5000 ok, 0 err)  | elapsed 734.2s
[lp8k] 6,000/6,745  (+6000 ok, 0 err)  | elapsed 882.8s
[lp8k] 6,745/6,745  (+6745 ok, 0 err)  | elapsed 996.6s
[lp8k] finished: +6745 new, 0 errors | total elapsed 996.6s
[sum] opus1

### **3. Source Labeling Helper**
We define a helper function to standardize source labels (e.g., `real_commonvoice`, `fake_tts`) based on file paths. This ensures consistency across all metadata files.

In [ ]:
def src_from_path(p: str) -> str:
    p = str(p)
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    if "/raw/fake_vc"               in p: return "fake_vc"     # <— add this later
    return "other"

### **4. Attack Metadata Cleaning (Robust)**
We perform a deep clean of the attack metadata by cross-referencing the `master_real.csv` and `master_fake.csv` files. This ensures that every attacked file (even those recovered via backfill) inherits the correct label, speaker ID, and text from its original parent file.

In [ ]:
# === Clean + label attacks.clean.csv using master manifests (robust) ===
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = "/content/drive/MyDrive/hindi_dfake"
META = Path(ROOT)/"metadata"

ATT_CLEAN = META/"attacks.clean.csv"
REAL_CSV  = META/"master_real.csv"
FAKE_CSV  = META/"master_fake.csv"

assert ATT_CLEAN.exists(), f"Missing: {ATT_CLEAN}"
assert REAL_CSV.exists() and FAKE_CSV.exists(), "Missing master_real.csv or master_fake.csv"

# ---- helpers ----
def src_from_path(p: str) -> str:
    p = str(p)
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    if "/raw/fake_vc"               in p: return "fake_vc"
    return "other"

from pathlib import Path as _P
def orig_stem_from_attacked(path: str, attack: str) -> str:
    s = _P(path).stem
    suf = f".{attack}"
    return s[:-len(suf)] if s.endswith(suf) else s

# ---- load and normalize attacks.clean.csv ----
dfA = pd.read_csv(ATT_CLEAN)
# ensure expected cols exist; keep current values in *_cur to avoid suffix mess after merge
for c in ["parent_utt","speaker_id","gender","text","label","source","fake_type","duration","attack","codec","bitrate","path"]:
    if c not in dfA.columns:
        dfA[c] = "" if c != "duration" else 0.0
rename_map = {
    "parent_utt":"parent_utt_cur",
    "speaker_id":"speaker_id_cur",
    "gender":"gender_cur",
    "text":"text_cur",
    "label":"label_cur",
    "source":"source_cur",
}
dfA = dfA.rename(columns=rename_map)
dfA["duration"] = pd.to_numeric(dfA["duration"], errors="coerce").fillna(0.0)
dfA["orig_stem"] = [orig_stem_from_attacked(p, a) for p, a in zip(dfA["path"], dfA["attack"])]

# ---- build stem->meta map from masters ----
def build_map(df_in, parent_label):
    rows = []
    for _, r in df_in.iterrows():
        rp = str(r.get("path",""))
        rows.append({
            "stem": _P(rp).stem,
            "m_label": parent_label,
            "m_source": src_from_path(rp),
            "m_fake_type": str(r.get("fake_type","")) if parent_label=="fake" else "",
            "m_utt": str(r.get("utt_id","")),
            "m_speaker": str(r.get("speaker_id","")),
            "m_gender": str(r.get("gender","")),
            "m_text": str(r.get("text","")),
        })
    return pd.DataFrame(rows)

dfR = pd.read_csv(REAL_CSV)
dfF = pd.read_csv(FAKE_CSV)

dfMap = pd.concat([build_map(dfR, "real"), build_map(dfF, "fake")], ignore_index=True)
dfMap = dfMap.drop_duplicates(subset=["stem"], keep="first")

# ---- merge attacks with map by original stem ----
dfM = dfA.merge(dfMap, how="left", left_on="orig_stem", right_on="stem")

# ---- fix label -> only {'real','fake'} ----
lab_cur = dfM["label_cur"].astype(str).str.lower()
lab_map = dfM["m_label"].astype(str).str.lower()
ft_map  = dfM["m_fake_type"].astype(str)

dfM["label"] = np.where(
    lab_cur.isin({"real","fake"}), lab_cur,
    np.where(lab_map.isin({"real","fake"}), lab_map,
             np.where(ft_map != "", "fake", ""))
)

# ---- fix source (prefer current, then map, then infer from path) ----
src_cur = dfM["source_cur"].astype(str)
src_final = src_cur.where(src_cur != "", dfM["m_source"])
src_final = src_final.where(src_final != "", dfM["path"].astype(str).map(src_from_path))
dfM["source"] = src_final

# ---- force fake_type for these rows ----
dfM["fake_type"] = "channel_attack"

# ---- backfill parent_utt/speaker/gender/text ----
def coalesce(cur, mapped):
    cur = cur.astype(str)
    mapped = mapped.astype(str)
    return np.where(cur != "", cur, mapped)

dfM["parent_utt"] = coalesce(dfM.get("parent_utt_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_utt", pd.Series([""]*len(dfM))))
dfM["speaker_id"] = coalesce(dfM.get("speaker_id_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_speaker", pd.Series([""]*len(dfM))))
dfM["gender"]     = coalesce(dfM.get("gender_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_gender", pd.Series([""]*len(dfM))))
dfM["text"]       = coalesce(dfM.get("text_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_text", pd.Series([""]*len(dfM))))

# ---- final tidy dataframe ----
out_cols = ["parent_utt","path","duration","speaker_id","gender","text","label",
            "fake_type","attack","codec","bitrate","source"]
df_out = dfM[out_cols].copy()

OUT = META/"attacks.labeled.csv"
df_out.to_csv(OUT, index=False)
print(f"Saved: {OUT} | rows={len(df_out)}")

# ---- quick summaries ----
def hours_series(s: pd.Series) -> float:
    return float(pd.to_numeric(s, errors="coerce").fillna(0.0).sum()/3600.0)

print("\nLabel counts:")
print(df_out["label"].value_counts(dropna=False).to_string())

print("\nBy attack × label:")
print(
    df_out.groupby(["attack","label"], dropna=False)
          .agg(files=("path","count"), hours=("duration", hours_series))
          .reset_index()
          .sort_values(["attack","label"])
          .to_string(index=False)
)

Saved: /content/drive/MyDrive/hindi_dfake/metadata/attacks.labeled.csv | rows=45093

Label counts:
label
real    23658
fake    21435

By attack × label:
attack label  files     hours
  lp8k  fake   9567 22.377793
  lp8k  real  10336 21.163750
opus16  fake  11868 27.002213
opus16  real  13322 29.203304
